In [9]:
import os.path
%run 0-0-common.ipynb

In [10]:
%%time

# subs = ["realasians", "KoreanHotties", "prettyasiangirls"]
subs = ["mildlypenis"]

extant_data: pd.DataFrame = pd.read_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system, schema=schema)

distinct_data: pd.DataFrame = extant_data.drop_duplicates(subset=['id', 'title']).reset_index(drop=True)

target_frame: pd.DataFrame = distinct_data[distinct_data['subreddit'].isin(subs)]

sorted_target_frame = target_frame.sort_values(by=['title'], ascending=False)

display(sorted_target_frame)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated


CPU times: total: 375 ms
Wall time: 1.39 s


In [11]:
%%time

known_ids = set(sorted_target_frame['id'].to_list())

display(len(known_ids))

0

CPU times: total: 0 ns
Wall time: 2 ms


In [12]:
%%time

reddit: praw.Reddit = praw.Reddit(client_id='5hVavL0PIRyM_1JSvqT6UQ', client_secret='BjD2kS3WNLnJc59RKY-JJUuc_Z9-JA',
								  user_agent='script:%(bot_name)s:v%(bot_version)s (by /u/%(bot_author)s)')

sub_data = reddit.subreddit("+".join(subs)).new(limit=None)

reddit_ids = list(sub_data)

Version 7.6.1 of praw is outdated. Version 7.7.0 was released Saturday February 25, 2023.


CPU times: total: 281 ms
Wall time: 1min 44s


In [13]:
%%time

initial = pd.DataFrame({}, columns=schema.names)

for elem in reddit_ids:
	if elem.id not in known_ids:
		record = {
			'id': elem.id,
			'subreddit': elem.subreddit,
			'author': elem.author,
			'title': elem.title,
			'caption': "",
			'hash': "",
			'permalink': elem.permalink,
			'original_url': elem.url,
			'image_name': "",
			'path': "",
			'thumbnail_path': "",
			'exists': False,
			'curated': False
		}
		initial = initial.append(record, ignore_index=True)
	else:
		continue

display(initial)

<timed exec>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:20: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
<timed exec>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:20: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
<timed exec>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:20: Fut

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,129jown,mildlypenis,mellywheats,Sourdough English muffins,,,/r/mildlypenis/comments/129jown/sourdough_engl...,https://i.redd.it/kt7wk38x0fra1.jpg,,,,False,False
1,129j6zk,mildlypenis,Aldous_Hoaxley,This jalapeno found at the supermarket,,,/r/mildlypenis/comments/129j6zk/this_jalapeno_...,https://i.redd.it/ztd9j3e1ogra1.jpg,,,,False,False
2,129iyab,mildlypenis,PhoenixBorealis,Kidney stones are really rough 😨,,,/r/mildlypenis/comments/129iyab/kidney_stones_...,https://i.redd.it/i3zaj3wu3ira1.jpg,,,,False,False
3,128rjm1,mildlypenis,itsboobthirty,Casually walking around a plant nursery and sa...,,,/r/mildlypenis/comments/128rjm1/casually_walki...,https://i.redd.it/lfw3hqdjecra1.jpg,,,,False,False
4,128z9kt,mildlypenis,Cole_A,These Easter decorations are getting out of hand,,,/r/mildlypenis/comments/128z9kt/these_easter_d...,https://i.imgur.com/bwEMZ9O.jpg,,,,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
990,11oz7fi,mildlypenis,Extra-Lie489,Lenis,,,/r/mildlypenis/comments/11oz7fi/lenis/,https://i.redd.it/pskj3q0ao8na1.jpg,,,,False,False
991,11oz73d,mildlypenis,DeimosLV,My friend is making a stuffed animal and this ...,,,/r/mildlypenis/comments/11oz73d/my_friend_is_m...,https://i.redd.it/gemqhq67o8na1.jpg,,,,False,False
992,11oyzm3,mildlypenis,Mymemesareswell,This weird Mickey keychain hand,,,/r/mildlypenis/comments/11oyzm3/this_weird_mic...,https://i.redd.it/vrivt8rom8na1.jpg,,,,False,False
993,11oyvx1,mildlypenis,LucindaBobinda,Parking lot pp,,,/r/mildlypenis/comments/11oyvx1/parking_lot_pp/,https://i.redd.it/w1vebizwl8na1.jpg,,,,False,False


CPU times: total: 1.84 s
Wall time: 1.81 s


In [14]:
%%time

records = initial.to_dict('records')

with tqdm(iterable=records, total=len(records), desc="global") as pbar:
	for item in records:
		pbar.update(1)
		if item['original_url'].endswith('.jpg') or item['original_url'].endswith('.png'):
			image_path, image_thumbnail_path, md5 = fetch_image(item['subreddit'], item['original_url'], list(extant_data['hash']))
			item['hash'] = md5
			item['image_name'] = f"{md5}.jpg"
			item['path'] = f"{image_path}"
			item['thumbnail_path'] = f"{image_thumbnail_path}"
			item['exists'] = True
		else:
			continue

global:   2%|▏         | 19/995 [00:02<02:22,  6.84it/s]

cannot write mode RGBA as JPEG


global:   3%|▎         | 32/995 [00:04<02:35,  6.20it/s]

cannot write mode RGBA as JPEG


global:   4%|▍         | 40/995 [00:05<01:52,  8.50it/s]

...

cannot write mode RGBA as JPEG


global:  99%|█████████▉| 983/995 [02:22<00:01,  8.16it/s]

cannot write mode RGBA as JPEG


global: 100%|██████████| 995/995 [02:24<00:00,  6.90it/s]

CPU times: total: 28.3 s
Wall time: 2min 24s


In [15]:
%%time

updated = pd.DataFrame(data=records, columns=schema.names)
display(updated)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,129jown,mildlypenis,mellywheats,Sourdough English muffins,,52e1039b4e930e5c3f879309bc626308,/r/mildlypenis/comments/129jown/sourdough_engl...,https://i.redd.it/kt7wk38x0fra1.jpg,52e1039b4e930e5c3f879309bc626308.jpg,D:\data\images\mildlypenis\52e1039b4e930e5c3f8...,D:\data\images\mildlypenis\thumbnail\52e1039b4...,True,False
1,129j6zk,mildlypenis,Aldous_Hoaxley,This jalapeno found at the supermarket,,fdb8066d1bffb12361334642d591e7f5,/r/mildlypenis/comments/129j6zk/this_jalapeno_...,https://i.redd.it/ztd9j3e1ogra1.jpg,fdb8066d1bffb12361334642d591e7f5.jpg,D:\data\images\mildlypenis\fdb8066d1bffb123613...,D:\data\images\mildlypenis\thumbnail\fdb8066d1...,True,False
2,129iyab,mildlypenis,PhoenixBorealis,Kidney stones are really rough 😨,,a03a3c52ef9aeac8bc56caff418feb9a,/r/mildlypenis/comments/129iyab/kidney_stones_...,https://i.redd.it/i3zaj3wu3ira1.jpg,a03a3c52ef9aeac8bc56caff418feb9a.jpg,D:\data\images\mildlypenis\a03a3c52ef9aeac8bc5...,D:\data\images\mildlypenis\thumbnail\a03a3c52e...,True,False
3,128rjm1,mildlypenis,itsboobthirty,Casually walking around a plant nursery and sa...,,ba3d4803c189dfe3d4b4ec818ef2b508,/r/mildlypenis/comments/128rjm1/casually_walki...,https://i.redd.it/lfw3hqdjecra1.jpg,ba3d4803c189dfe3d4b4ec818ef2b508.jpg,D:\data\images\mildlypenis\ba3d4803c189dfe3d4b...,D:\data\images\mildlypenis\thumbnail\ba3d4803c...,True,False
4,128z9kt,mildlypenis,Cole_A,These Easter decorations are getting out of hand,,f50e250ad34c47e94c13942d3e0a7153,/r/mildlypenis/comments/128z9kt/these_easter_d...,https://i.imgur.com/bwEMZ9O.jpg,f50e250ad34c47e94c13942d3e0a7153.jpg,D:\data\images\mildlypenis\f50e250ad34c47e94c1...,D:\data\images\mildlypenis\thumbnail\f50e250ad...,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
990,11oz7fi,mildlypenis,Extra-Lie489,Lenis,,7604c3e55ce8dbf238520683d6e71f89,/r/mildlypenis/comments/11oz7fi/lenis/,https://i.redd.it/pskj3q0ao8na1.jpg,7604c3e55ce8dbf238520683d6e71f89.jpg,D:\data\images\mildlypenis\7604c3e55ce8dbf2385...,D:\data\images\mildlypenis\thumbnail\7604c3e55...,True,False
991,11oz73d,mildlypenis,DeimosLV,My friend is making a stuffed animal and this ...,,389f8aa0fb191bdeba08c8d2cfff8ce2,/r/mildlypenis/comments/11oz73d/my_friend_is_m...,https://i.redd.it/gemqhq67o8na1.jpg,389f8aa0fb191bdeba08c8d2cfff8ce2.jpg,D:\data\images\mildlypenis\389f8aa0fb191bdeba0...,D:\data\images\mildlypenis\thumbnail\389f8aa0f...,True,False
992,11oyzm3,mildlypenis,Mymemesareswell,This weird Mickey keychain hand,,87563ac0c8ddeeb0ce47bc915b99308b,/r/mildlypenis/comments/11oyzm3/this_weird_mic...,https://i.redd.it/vrivt8rom8na1.jpg,87563ac0c8ddeeb0ce47bc915b99308b.jpg,D:\data\images\mildlypenis\87563ac0c8ddeeb0ce4...,D:\data\images\mildlypenis\thumbnail\87563ac0c...,True,False
993,11oyvx1,mildlypenis,LucindaBobinda,Parking lot pp,,88b596be6de959bd29eed4e8946bb476,/r/mildlypenis/comments/11oyvx1/parking_lot_pp/,https://i.redd.it/w1vebizwl8na1.jpg,88b596be6de959bd29eed4e8946bb476.jpg,D:\data\images\mildlypenis\88b596be6de959bd29e...,D:\data\images\mildlypenis\thumbnail\88b596be6...,True,False


CPU times: total: 15.6 ms
Wall time: 18 ms


In [23]:
%%time

from shared_code.utility.scripts.blip_caption import BlipCaption

blip = BlipCaption(1)

CPU times: total: 25.4 s
Wall time: 23.7 s
CPU times: total: 28.5 s
Wall time: 26.5 s


In [24]:
%%time

def caption_image(x: object, blip_captioning: BlipCaption) -> str:
	try:
		path = x['path']
		exists_image = bool(x['exists'])
		resulting_caption = x['caption']

		if not os.path.exists(path):
			return ""

		if not exists_image:
			return ""

		if len(resulting_caption) > 5:
			return resulting_caption

		resulting_caption = blip_captioning.caption_image(path)
		return resulting_caption

	except Exception as e:
		print(e)
		return ""

CPU times: total: 0 ns
Wall time: 0 ns
CPU times: total: 0 ns
Wall time: 0 ns


In [25]:
%%time

ddf = dd.from_pandas(updated, npartitions=3)

new_entries_final = updated.copy()


new_entries_final['caption'] = ddf.apply(lambda x: caption_image(x, blip), meta=('str', object), axis=1).compute()

global:   0%|          | 0/995 [00:00<?, ?it/s]D:\workspaces\General\venv\lib\site-packages\transformers\generation\utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
D:\workspaces\General\venv\lib\site-packages\transformers\generation\utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
D:\workspaces\General\venv\lib\site-packages\transformers\generation\utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of 

CPU times: total: 34min 36s
Wall time: 14min 48s



global:   0%|          | 0/995 [00:00<?, ?it/s]

Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cuda:1! (when checking argument for argument weight in method wrapper_CUDA__cudnn_convolution)
Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cuda:1! (when checking argument for argument weight in method wrapper_CUDA__cudnn_convolution)
Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cuda:1! (when checking argument for argument weight in method wrapper_CUDA__cudnn_convolution)
Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cuda:1! (when checking argument for argument weight in method wrapper_CUDA__cudnn_convolution)
Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cuda:1! (when checking argument for argument weight in method wrapper_CUDA__cudnn_convolution)
Expected all tensors to be on the same device, but found at least two devic

global:   0%|          | 0/995 [01:11<?, ?it/s]

Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cuda:1! (when checking argument for argument weight in method wrapper_CUDA__cudnn_convolution)


Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cuda:1! (when checking argument for argument weight in method wrapper_CUDA__cudnn_convolution)
Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cuda:1! (when checking argument for argument weight in method wrapper_CUDA__cudnn_convolution)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ D:\workspaces\General\venv\lib\site-packages\IPython\core\magics\execution.py:1316 in time       │
│                                                                                                  │
│   1313 │   │   else:                                                                             │
│   1314 │   │   │   st = clock2()                                                                 │
│   1315 │   │   │   try:                                                                          │
│ ❱ 1316 │   │   │   │   exec(code, glob, local_ns)                                                │
│   1317 │   │   │   │   out=None                                                                  │
│   1318 │   │   │   │   # multi-line %%time case                                                  │
│   1319 │   │   │   │   if expr_val is not None:                                                  │
│ in <module>                                                                                      │
│                                                                                                  │
│ D:\workspaces\General\venv\lib\site-packages\dask\base.py:314 in compute                         │
│                                                                                                  │
│    311 │   │   --------                                                                          │
│    312 │   │   dask.base.compute                                                                 │
│    313 │   │   """                                                                               │
│ ❱  314 │   │   (result,) = compute(self, traverse=False, **kwargs)                               │
│    315 │   │   return result                                                                     │
│    316 │                                                                                         │
│    317 │   def __await__(self):                                                                  │
│                                                                                                  │
│ D:\workspaces\General\venv\lib\site-packages\dask\base.py:599 in compute                         │
│                                                                                                  │
│    596 │   │   keys.append(x.__dask_keys__())                                                    │
│    597 │   │   postcomputes.append(x.__dask_postcompute__())                                     │
│    598 │                                                                                         │
│ ❱  599 │   results = schedule(dsk, keys, **kwargs)                                               │
│    600 │   return repack([f(r, *a) for r, (f, a) in zip(results, postcomputes)])                 │
│    601                                                                                           │
│    602                                                                                           │
│                                                                                                  │
│ D:\workspaces\General\venv\lib\site-packages\dask\threaded.py:89 in get                          │
│                                                                                                  │
│    86 │   │   elif isinstance(pool, multiprocessing.pool.Pool):                                  │
│    87 │   │   │   pool = MultiprocessingPoolExecutor(pool)                                       │
│    88 │                                                                                          │
│ ❱  89 │   results = get_async(                                                                   │
│    90 │   │   pool.submit,                                                                       │
│    91 │   │   pool._max_workers,                           

Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cuda:1! (when checking argument for argument weight in method wrapper_CUDA__cudnn_convolution)Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cuda:1! (when checking argument for argument weight in method wrapper_CUDA__cudnn_convolution)

Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cuda:1! (when checking argument for argument weight in method wrapper_CUDA__cudnn_convolution)


In [26]:
%%time

display(new_entries_final)

appended_extant_data = extant_data.append(new_entries_final, ignore_index=True)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,129jown,mildlypenis,mellywheats,Sourdough English muffins,,52e1039b4e930e5c3f879309bc626308,/r/mildlypenis/comments/129jown/sourdough_engl...,https://i.redd.it/kt7wk38x0fra1.jpg,52e1039b4e930e5c3f879309bc626308.jpg,D:\data\images\mildlypenis\52e1039b4e930e5c3f8...,D:\data\images\mildlypenis\thumbnail\52e1039b4...,True,False
1,129j6zk,mildlypenis,Aldous_Hoaxley,This jalapeno found at the supermarket,,fdb8066d1bffb12361334642d591e7f5,/r/mildlypenis/comments/129j6zk/this_jalapeno_...,https://i.redd.it/ztd9j3e1ogra1.jpg,fdb8066d1bffb12361334642d591e7f5.jpg,D:\data\images\mildlypenis\fdb8066d1bffb123613...,D:\data\images\mildlypenis\thumbnail\fdb8066d1...,True,False
2,129iyab,mildlypenis,PhoenixBorealis,Kidney stones are really rough 😨,,a03a3c52ef9aeac8bc56caff418feb9a,/r/mildlypenis/comments/129iyab/kidney_stones_...,https://i.redd.it/i3zaj3wu3ira1.jpg,a03a3c52ef9aeac8bc56caff418feb9a.jpg,D:\data\images\mildlypenis\a03a3c52ef9aeac8bc5...,D:\data\images\mildlypenis\thumbnail\a03a3c52e...,True,False
3,128rjm1,mildlypenis,itsboobthirty,Casually walking around a plant nursery and sa...,,ba3d4803c189dfe3d4b4ec818ef2b508,/r/mildlypenis/comments/128rjm1/casually_walki...,https://i.redd.it/lfw3hqdjecra1.jpg,ba3d4803c189dfe3d4b4ec818ef2b508.jpg,D:\data\images\mildlypenis\ba3d4803c189dfe3d4b...,D:\data\images\mildlypenis\thumbnail\ba3d4803c...,True,False
4,128z9kt,mildlypenis,Cole_A,These Easter decorations are getting out of hand,,f50e250ad34c47e94c13942d3e0a7153,/r/mildlypenis/comments/128z9kt/these_easter_d...,https://i.imgur.com/bwEMZ9O.jpg,f50e250ad34c47e94c13942d3e0a7153.jpg,D:\data\images\mildlypenis\f50e250ad34c47e94c1...,D:\data\images\mildlypenis\thumbnail\f50e250ad...,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
990,11oz7fi,mildlypenis,Extra-Lie489,Lenis,,7604c3e55ce8dbf238520683d6e71f89,/r/mildlypenis/comments/11oz7fi/lenis/,https://i.redd.it/pskj3q0ao8na1.jpg,7604c3e55ce8dbf238520683d6e71f89.jpg,D:\data\images\mildlypenis\7604c3e55ce8dbf2385...,D:\data\images\mildlypenis\thumbnail\7604c3e55...,True,False
991,11oz73d,mildlypenis,DeimosLV,My friend is making a stuffed animal and this ...,,389f8aa0fb191bdeba08c8d2cfff8ce2,/r/mildlypenis/comments/11oz73d/my_friend_is_m...,https://i.redd.it/gemqhq67o8na1.jpg,389f8aa0fb191bdeba08c8d2cfff8ce2.jpg,D:\data\images\mildlypenis\389f8aa0fb191bdeba0...,D:\data\images\mildlypenis\thumbnail\389f8aa0f...,True,False
992,11oyzm3,mildlypenis,Mymemesareswell,This weird Mickey keychain hand,,87563ac0c8ddeeb0ce47bc915b99308b,/r/mildlypenis/comments/11oyzm3/this_weird_mic...,https://i.redd.it/vrivt8rom8na1.jpg,87563ac0c8ddeeb0ce47bc915b99308b.jpg,D:\data\images\mildlypenis\87563ac0c8ddeeb0ce4...,D:\data\images\mildlypenis\thumbnail\87563ac0c...,True,False
993,11oyvx1,mildlypenis,LucindaBobinda,Parking lot pp,,88b596be6de959bd29eed4e8946bb476,/r/mildlypenis/comments/11oyvx1/parking_lot_pp/,https://i.redd.it/w1vebizwl8na1.jpg,88b596be6de959bd29eed4e8946bb476.jpg,D:\data\images\mildlypenis\88b596be6de959bd29e...,D:\data\images\mildlypenis\thumbnail\88b596be6...,True,False


CPU times: total: 438 ms
Wall time: 40.1 ms


<timed exec>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


In [27]:
%%time

appended_extant_data.reset_index().drop('index', axis=1, inplace=True)

display(appended_extant_data)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,1000cej,CityPorn,OtterlyFoxy,New York in the fog,cars are parked on the side of the road in the...,7a8d96e378c15c8ab8440ac311f12c11,/r/CityPorn/comments/1000cej/new_york_in_the_fog/,https://i.redd.it/4emw5uldib9a1.jpg,4emw5uldib9a1.jpg,D:\data\images\CityPorn\4emw5uldib9a1.jpg,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,True,False
1,1000glf,AmIhotAF,toolate_sharkbait,Just looking for entertainment,blonde woman with blonde hair and tattoos on h...,e554c1ed7ffa2740436ac082068b2824,/r/AmIhotAF/comments/1000glf/just_looking_for_...,https://i.redd.it/4xyb1vgbjb9a1.jpg,4xyb1vgbjb9a1.jpg,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,True,False
2,1000j1n,greentext,trent8051,Anon wants Elon cut,a man with a beard and a beard sitting in fron...,1dec3dabb5e46cde01855d06089c287a,/r/greentext/comments/1000j1n/anon_wants_elon_...,https://i.redd.it/3mewbe0wjb9a1.jpg,3mewbe0wjb9a1.jpg,D:\data\images\greentext\3mewbe0wjb9a1.jpg,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,True,False
3,1000mjs,spaceporn,MorningStar_imangi,Northern Lights above Lofoten,a view of a view of a large green and purple a...,2c39ce1290fba541abd0b004b09da6b2,/r/spaceporn/comments/1000mjs/northern_lights_...,https://i.redd.it/7s5aafaqkb9a1.jpg,7s5aafaqkb9a1.jpg,/data/images/spaceporn/7s5aafaqkb9a1.jpg,/data/images/spaceporn/thumbnail/7s5aafaqkb9a1...,True,False
4,1000qpd,spaceporn,MorningStar_imangi,Viking Lights,a scene of a boat is sitting on the shore of a...,0f72de47c69ff50eca5fa3990215f4ac,/r/spaceporn/comments/1000qpd/viking_lights/,https://i.redd.it/abojw7lqlb9a1.jpg,abojw7lqlb9a1.jpg,/data/images/spaceporn/abojw7lqlb9a1.jpg,/data/images/spaceporn/thumbnail/abojw7lqlb9a1...,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29600,11oz7fi,mildlypenis,Extra-Lie489,Lenis,,7604c3e55ce8dbf238520683d6e71f89,/r/mildlypenis/comments/11oz7fi/lenis/,https://i.redd.it/pskj3q0ao8na1.jpg,7604c3e55ce8dbf238520683d6e71f89.jpg,D:\data\images\mildlypenis\7604c3e55ce8dbf2385...,D:\data\images\mildlypenis\thumbnail\7604c3e55...,True,False
29601,11oz73d,mildlypenis,DeimosLV,My friend is making a stuffed animal and this ...,,389f8aa0fb191bdeba08c8d2cfff8ce2,/r/mildlypenis/comments/11oz73d/my_friend_is_m...,https://i.redd.it/gemqhq67o8na1.jpg,389f8aa0fb191bdeba08c8d2cfff8ce2.jpg,D:\data\images\mildlypenis\389f8aa0fb191bdeba0...,D:\data\images\mildlypenis\thumbnail\389f8aa0f...,True,False
29602,11oyzm3,mildlypenis,Mymemesareswell,This weird Mickey keychain hand,,87563ac0c8ddeeb0ce47bc915b99308b,/r/mildlypenis/comments/11oyzm3/this_weird_mic...,https://i.redd.it/vrivt8rom8na1.jpg,87563ac0c8ddeeb0ce47bc915b99308b.jpg,D:\data\images\mildlypenis\87563ac0c8ddeeb0ce4...,D:\data\images\mildlypenis\thumbnail\87563ac0c...,True,False
29603,11oyvx1,mildlypenis,LucindaBobinda,Parking lot pp,,88b596be6de959bd29eed4e8946bb476,/r/mildlypenis/comments/11oyvx1/parking_lot_pp/,https://i.redd.it/w1vebizwl8na1.jpg,88b596be6de959bd29eed4e8946bb476.jpg,D:\data\images\mildlypenis\88b596be6de959bd29e...,D:\data\images\mildlypenis\thumbnail\88b596be6...,True,False


CPU times: total: 328 ms
Wall time: 82.1 ms


In [21]:
%%time

appended_extant_data.to_csv("processed_raw_data.csv", index=False)

final = pd.read_csv("processed_raw_data.csv")

final.to_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system, schema=schema)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ D:\workspaces\General\venv\lib\site-packages\IPython\core\magics\execution.py:1316 in time       │
│                                                                                                  │
│   1313 │   │   else:                                                                             │
│   1314 │   │   │   st = clock2()                                                                 │
│   1315 │   │   │   try:                                                                          │
│ ❱ 1316 │   │   │   │   exec(code, glob, local_ns)                                                │
│   1317 │   │   │   │   out=None                                                                  │
│   1318 │   │   │   │   # multi-line %%time case                                                  │
│   1319 │   │   │   │   if expr_val is not None:                                                  │
│ in <module>                                                                                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'appended_extant_data' is not defined

CPU times: total: 1.25 s
Wall time: 3.82 s


In [22]:
%%time

final = pd.read_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system)

display(final)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ D:\workspaces\General\venv\lib\site-packages\IPython\core\magics\execution.py:1316 in time       │
│                                                                                                  │
│   1313 │   │   else:                                                                             │
│   1314 │   │   │   st = clock2()                                                                 │
│   1315 │   │   │   try:                                                                          │
│ ❱ 1316 │   │   │   │   exec(code, glob, local_ns)                                                │
│   1317 │   │   │   │   out=None                                                                  │
│   1318 │   │   │   │   # multi-line %%time case                                                  │
│   1319 │   │   │   │   if expr_val is not None:                                                  │
│ in <module>                                                                                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'pd' is not defined

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,1000cej,CityPorn,OtterlyFoxy,New York in the fog,cars are parked on the side of the road in the...,7a8d96e378c15c8ab8440ac311f12c11,/r/CityPorn/comments/1000cej/new_york_in_the_fog/,https://i.redd.it/4emw5uldib9a1.jpg,4emw5uldib9a1.jpg,D:\data\images\CityPorn\4emw5uldib9a1.jpg,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,True,False
1,1000glf,AmIhotAF,toolate_sharkbait,Just looking for entertainment,blonde woman with blonde hair and tattoos on h...,e554c1ed7ffa2740436ac082068b2824,/r/AmIhotAF/comments/1000glf/just_looking_for_...,https://i.redd.it/4xyb1vgbjb9a1.jpg,4xyb1vgbjb9a1.jpg,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,True,False
2,1000j1n,greentext,trent8051,Anon wants Elon cut,a man with a beard and a beard sitting in fron...,1dec3dabb5e46cde01855d06089c287a,/r/greentext/comments/1000j1n/anon_wants_elon_...,https://i.redd.it/3mewbe0wjb9a1.jpg,3mewbe0wjb9a1.jpg,D:\data\images\greentext\3mewbe0wjb9a1.jpg,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,True,False
3,1000mjs,spaceporn,MorningStar_imangi,Northern Lights above Lofoten,a view of a view of a large green and purple a...,2c39ce1290fba541abd0b004b09da6b2,/r/spaceporn/comments/1000mjs/northern_lights_...,https://i.redd.it/7s5aafaqkb9a1.jpg,7s5aafaqkb9a1.jpg,/data/images/spaceporn/7s5aafaqkb9a1.jpg,/data/images/spaceporn/thumbnail/7s5aafaqkb9a1...,True,False
4,1000qpd,spaceporn,MorningStar_imangi,Viking Lights,a scene of a boat is sitting on the shore of a...,0f72de47c69ff50eca5fa3990215f4ac,/r/spaceporn/comments/1000qpd/viking_lights/,https://i.redd.it/abojw7lqlb9a1.jpg,abojw7lqlb9a1.jpg,/data/images/spaceporn/abojw7lqlb9a1.jpg,/data/images/spaceporn/thumbnail/abojw7lqlb9a1...,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29600,11oz7fi,mildlypenis,Extra-Lie489,Lenis,there is a small piece of metal with two small...,7604c3e55ce8dbf238520683d6e71f89,/r/mildlypenis/comments/11oz7fi/lenis/,https://i.redd.it/pskj3q0ao8na1.jpg,7604c3e55ce8dbf238520683d6e71f89.jpg,D:\data\images\mildlypenis\7604c3e55ce8dbf2385...,D:\data\images\mildlypenis\thumbnail\7604c3e55...,True,False
29601,11oz73d,mildlypenis,DeimosLV,My friend is making a stuffed animal and this ...,a close up of a blue knitted bagueth on a chair,389f8aa0fb191bdeba08c8d2cfff8ce2,/r/mildlypenis/comments/11oz73d/my_friend_is_m...,https://i.redd.it/gemqhq67o8na1.jpg,389f8aa0fb191bdeba08c8d2cfff8ce2.jpg,D:\data\images\mildlypenis\389f8aa0fb191bdeba0...,D:\data\images\mildlypenis\thumbnail\389f8aa0f...,True,False
29602,11oyzm3,mildlypenis,Mymemesareswell,This weird Mickey keychain hand,someone is holding a key chain with a mickey m...,87563ac0c8ddeeb0ce47bc915b99308b,/r/mildlypenis/comments/11oyzm3/this_weird_mic...,https://i.redd.it/vrivt8rom8na1.jpg,87563ac0c8ddeeb0ce47bc915b99308b.jpg,D:\data\images\mildlypenis\87563ac0c8ddeeb0ce4...,D:\data\images\mildlypenis\thumbnail\87563ac0c...,True,False
29603,11oyvx1,mildlypenis,LucindaBobinda,Parking lot pp,a close up of a black and white photo of a bla...,88b596be6de959bd29eed4e8946bb476,/r/mildlypenis/comments/11oyvx1/parking_lot_pp/,https://i.redd.it/w1vebizwl8na1.jpg,88b596be6de959bd29eed4e8946bb476.jpg,D:\data\images\mildlypenis\88b596be6de959bd29e...,D:\data\images\mildlypenis\thumbnail\88b596be6...,True,False


CPU times: total: 203 ms
Wall time: 836 ms
